## Assignment 2: Data Cleansing <br>

Name: Manmeet Singh <br>
Id: 30749476

In [ ]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from sklearn.linear_model import LinearRegression
from datetime import datetime
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt

### Methodology<br>

I have use non graphical EDA to analyze the datasets except for outlier data. <br>
I will be explaning my Methodology step by step for each cell for better understanding.<br>
Please note for few cells output has been printed for EDA and explanation.

### Read data file for dirty data and Warehouse

In [ ]:
dirty_data = pd.read_csv('30749476_dirty_data.csv')

warehouse_data = pd.read_csv('warehouses.csv')
# convert dataframe to dictionary
warehouse_dict = warehouse_data.to_dict()
# dictionary of Nickolson with it's coordinate 
Nickolson = {warehouse_dict['names'][0]:(warehouse_dict['lat'][0],warehouse_dict['lon'][0])}
# dictionary of Thompson with it's coordinate
Thompson = {warehouse_dict['names'][1]:(warehouse_dict['lat'][1],warehouse_dict['lon'][1])}
# dictionary of Bakers with it's coordinate
Bakers = {warehouse_dict['names'][2]:(warehouse_dict['lat'][2],warehouse_dict['lon'][2])}

#### Step 1<br>
**Check the information, it provides how many entries, and columns in dataset. Provide the type of each column and if any null values.**<br>
As we can see there are 500 entries and 16 columms with no null values

In [ ]:
dirty_data.info()

#### Step 2 <br>
**Let's perform some more analysis**<br> 
1. From the description below for numeric columns, we can see the min, max, std, count for all numeric columns.<br>
2. WE can see that customer_lat column has max value of 145.007284 and customer_long has min value -37.826093, which does not seem right.<br>
3. For other columns we can't see any such information, which for npow seems good, but can't be guaranteed. For them we will perform other type of analysis.

In [ ]:
dirty_data.describe()

#### Step 3<br>
**Let's perform analysis on categorical columns**<br>

1. Column date,nearest_warehouse,shopping_cart,season has some duplicate entries. We will determine if they are good or needs to be removed.<br>
2. customer_id also have 18 duplicate entries. We will analyze if they are repeated rows or different because it is possible same customer has made order more than once.

In [ ]:
dirty_data.describe(include=['O'])

In [ ]:
# dirty_data[dirty_data['customer_id'].duplicated()]

#### Step 4 <br>
**Get Unique values for Season Column**<br>
Below we can see it has 8 unique values, but same 4 season with lower case format.

In [ ]:
dirty_data['season'].unique()  # get unqiue values for season column

#### Step 5 <br>
**Get Unique values for Season Column**<br>
Below we can see it has 6 unique values, but same 3 warehouse with lower case format.

In [ ]:
dirty_data['nearest_warehouse'].unique() # # get unqiue values for nearest_warehouse column

#### Step 6<br>
**Replace the lower format elements with their respective correct format for nearest_warehouse and season column**

In [ ]:
# update the warehouse name to of same format
dirty_data['nearest_warehouse'] = dirty_data['nearest_warehouse'].replace({'thompson':'Thompson','nickolson':'Nickolson',
                                                                          'bakers': 'Bakers'})
# update the season name to of same format
dirty_data['season'] = dirty_data['season'].replace({'summer':'Summer','winter':'Winter',
                                                     'autumn': 'Autumn', 'spring':'Spring'})

#### Step 7 <br>
**Date column might have different formats, so creat a dateConversion fucntion which detect the date format using regex and convert the date finally to YYYY-mm-dd format. After that change the column type from object to datetime format**

In [ ]:
# function to get the format of date
def dateConversion(x):
    
    if re.match(r"^\d{1,2}/", x['date']):
        dateObj = datetime.strptime(x['date'],'%d/%m/%Y')
    try:
        if re.match(r'[0-9|/]+', x['date']):
            dateObj = datetime.strptime(x['date'],'%Y-%m-%d')
    except:
        try:
            if re.match(r'[0-9|/]+', x['date']):
                dateObj = datetime.strptime(x['date'],'%d-%m-%Y')
        except:
            dateObj = datetime.strptime(x['date'],'%Y-%d-%m')
    # convert the date to YYYY-mm-dd format
    date = dateObj.strftime('%Y-%m-%d')
    return date

# apply dateconversion function to date column of dirt data
dirty_data['date'] = dirty_data.apply(dateConversion, axis=1)
# change the format of date col from object to datetime
dirty_data['date'] = pd.to_datetime(dirty_data['date'])

#### Step 8: Customer coordinates<br>
From step 2 we know that customer_lat and customer_long have some incorrect values, they contain respectively each other value. To make it correct we will swap the values.

In [ ]:
# Swap the latitude and longitude values for customer_lat and customer_long columns where they have incorrect coordinates
dirty_data['customer_lat'],dirty_data['customer_long'] = np.where(dirty_data['customer_lat'] > 0, 
                                      (dirty_data['customer_long'],dirty_data['customer_lat']),
                                      (dirty_data['customer_lat'],dirty_data['customer_long'])) 

#### Step 9: Duplicate Entries<br>
1. From Step 2 we know customer_id have some duplicate entrie. Since these id's are unique we will check if any entry is repeated.<br>
2. Below We can see the dataframe with duplicate entries only<br>
3. No entry is repeated, so they are fair.<br>
4. For customer ID0846591613 (index 52,97), have same coordinates but different nearest warehouse. **It means it is possible that other customer might also have same issue. We will rectify it in next step, Step 10.**

In [ ]:
# get the dataframe for duplicate customer id
ids = dirty_data["customer_id"]
# sort data frame with customer_id
dirty_data[ids.isin(ids[ids.duplicated()])].sort_values(by=['customer_id'])

#### Step 10: Check correct Nearest Warehouse<br>
1. Issue found in **Step 9, part 4**. We will rectify it by following method: <br>
    a. Create a fucntion **"Kilometer"** to calculate distance from customer house to all warehouses.<br>
    b. Create a fucntion **"warehouse"**, we will get the coordinate of warehouses and customer. Check distance of each warehouse from customer home and take the minimum distance.<br>
    c. Pass a condition if calculated distance is equal to current distance, keep the current warehouse, if not then replace the current warehouse with update warehouse information.

In [ ]:
# fucntion to calculate distance between two coordinates
# referred to https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
def kilometers(lat1,lat2,long1,long2,Radius):
    longitude = long2 - long1
    latitude = lat2 - lat1
    # calculate arc
    arc = (sin(latitude/2))**2 + cos(lat1) * cos(lat2) * (sin(longitude/2))**2
    circumfrence = 2 * atan2(sqrt(arc), sqrt(1-arc))
    # distance calculation
    actual_dictance = Radius * circumfrence
    
    return round(actual_dictance,4)

def warehouse(x):
    # empty list
    kiloList = [] 
    Radius = 6378 # earth radius
    # customer latitude
    lat2 = radians(x['customer_lat'])
    # customer longitude
    long2 = radians(x['customer_long'])
    # warehouse list
    warehouseList = ['Nickolson','Thompson','Bakers']
    # initiate for loop
    for location in warehouseList:
        # if Nickolson warehouse
        if location == 'Nickolson':
            # get Nickolson coordinates
            lat1 = radians(Nickolson['Nickolson'][0])
            long1 = radians(Nickolson['Nickolson'][1])
            # calculate distance with customer location
            distance = kilometers(lat1,lat2,long1,long2,Radius)
            # append list
            kiloList.append(distance)
        # if Thompson warehouse
        if location == 'Thompson':
            # get Thompson coordinates
            lat1 = radians(Thompson['Thompson'][0])
            long1 = radians(Thompson['Thompson'][1])
             # calculate distance with customer location
            distance = kilometers(lat1,lat2,long1,long2,Radius)
            # append list
            kiloList.append(distance)

        if location == 'Bakers':
            # get Bakers coordinates
            lat1 = radians(Bakers['Bakers'][0])
            long1 = radians(Bakers['Bakers'][1])
            # calculate distance with customer location
            distance = kilometers(lat1,lat2,long1,long2,Radius)
            # append list
            kiloList.append(distance)
    # minimum value from the list
    min_distance = min(kiloList)
    # get index of minimum value
    ind = kiloList.index(min_distance)
    # get the warehouse based on the index found through minimum distance
    actual_warehouse = warehouseList[ind]
    
    if x['nearest_warehouse'] == actual_warehouse: # if calculated distnce is current, return current distance
        return x['nearest_warehouse']
    else:
        return actual_warehouse # else return calculated distance

# apply warehouse function on nearest_warehouse
dirty_data['nearest_warehouse'] = dirty_data.apply(warehouse,axis=1)

#### Step 11: Re check if issue found in step 9 rectified by step 10.<br>
We will check again if the issue of warehouse have been rectified for user ID0846591613 or not. We can see below, it has been rectified and other users also have correct information.

In [ ]:
# get the dataframe for duplicate customer id
ids = dirty_data["customer_id"]
# sort data frame with customer_id
dirty_data[ids.isin(ids[ids.duplicated()])].sort_values(by=['customer_id'])

#### Step 12: Check Total Order Price. <br>
**Similarly we will check for anomalies if any total order price is incorrect.**<br>
1. We cannot determine if total order price has any anomalies from summary. So we will perform following steps.<br>
    a. Create function **"order"** which calculate total order price from order price minus discounted price plus delivery charges.<br>
    b. Now, we check if current total price is equal to calculated total price, keep the current value else replace it with correct calculated value.


In [ ]:
# function to calculate total order price
def order(x):
    # total order price
    actual_order_price = x['order_price']*(1-x['coupon_discount']/100)+x['delivery_charges']
    # if current price equal calculated, keep current
    if actual_order_price == x['order_total']:
        return x['order_total']
    else:
        return actual_order_price # else update calculated value

dirty_data['order_total'] = dirty_data.apply(order, axis=1) # apply order fucntion on order_total col

Spring - the three transition months September, October and November.<br>
Summer - the three hottest months December, January and February.<br>
Autumn - the transition months March, April and May.<br>
Winter - the three coldest months June, July and August.<br>

#### Step 13: Check correct Season based on Month of Date. <br>
1. Since we have already corrected the format of season values in Step 6. Now we will check based on date if the current season is correct or not. <br>
2. First we get list of months for each season.<br>
3. Create function **"season"**, in this fucntion we will first fetch the month of the date.<br>
4. After that we will check in which season list this month belongs (Spring, Summer, Autumn, Winter).<br>
5. After gettting the season in above step, compare it with the current value, if same keep current value, else update it with new value found in above step.

In [ ]:
# defining months as list for all seasons
Spring = [9,10,11]
Summer = [12,1,2]
Autumn = [3,4,5]
Winter = [6,7,8]

In [ ]:
def season(x):
    # get the month from date column
    month = int(str(x['date']).split('-')[1])
    
    if month in Spring:
        # if current value is Spring, keep it, else update it.
        if x['season'] == 'Spring':
            return x['season']
        else:
            return 'Spring'
    elif month in Summer:
        # if current value is Summer, keep it, else update it.
        if x['season'] == 'Summer':
            return x['season']
        else:
            return 'Summer'
    elif month in Autumn:
        # if current value is Autumn, keep it, else update it.
        if x['season'] == 'Autumn':
            return x['season']
        else:
            return 'Autumn'
    elif month in Winter:
        # if current value is Winter, keep it, else update it.
        if x['season'] == 'Winter':
            return x['season']
        else:
            return 'Winter'
dirty_data['season'] = dirty_data.apply(season,axis=1) # apply season function on season col

#### Step 14. Get the brands from Shopping Cart<br>
In this step we will check if the items in shopping cart is less than, more than or equal to 10. Also if each item has different format or not.<br>
1. First get the value from each cell, convert the string of list to List format.<br>
2. Add each item in a list and finally take a set to get the unique values. We can see below we have 10 items with no different format or such.

In [ ]:
cart_list = []
for items in dirty_data['shopping_cart']:
    for item in eval(items):
        cart_list.append(item[0])
set(cart_list)

#### Step 15: Check distance to nearest warehouse.<br>
1. As we have rectified the nearest warehouse location for each customer. We need to rectify the distance as well.<br>
2. Create a function **"distance"**, now check which is the nearest warehouse to the customer based on the coordinates.<br>
3. If the calculated distance is equal to current value, keep current value, else replace it with distance calculated.

In [ ]:
def distance(x):
    Radius = 6378 # Earth Radius
    if x['nearest_warehouse'] == 'Nickolson':
        # if nearest warehouse is Nickolson, get its coordinate
        lat1 = radians(Nickolson['Nickolson'][0])
        long1 = radians(Nickolson['Nickolson'][1])
        
    if x['nearest_warehouse'] == 'Thompson':
        # if nearest warehouse is Thompson, get its coordinate
        lat1 = radians(Thompson['Thompson'][0])
        long1 = radians(Thompson['Thompson'][1])
    
    if x['nearest_warehouse'] == 'Bakers':
        # if nearest warehouse is Bakers, get its coordinate
        lat1 = radians(Bakers['Bakers'][0])
        long1 = radians(Bakers['Bakers'][1])
    # customer coordinates
    lat2 = radians(x['customer_lat'])
    long2 = radians(x['customer_long'])

    longitude = long2 - long1
    latitude = lat2 - lat1
    # arc calculation
    arc = (sin(latitude/2))**2 + cos(lat1) * cos(lat2) * (sin(longitude/2))**2
    circumfrence = 2 * atan2(sqrt(arc), sqrt(1-arc))
    # actual distance
    actual_dictance = Radius * circumfrence
    
    # if calculated distance is current distance, keep it, else update with calculated value
    if actual_dictance == x['distance_to_nearest_warehouse']:
        return x['distance_to_nearest_warehouse']
    else:
        return round(actual_dictance,4)

# apply distance function on distance_to_nearest_warehouse
dirty_data['distance_to_nearest_warehouse'] = dirty_data.apply(distance, axis=1)

#### Let's take the summary of numeric values again.<br>
We can see statistical values has been updated for cusstomer coordinates, and order_total.

In [ ]:
dirty_data.describe()

#### Let's take the summary of categorical values again.
¶
We can see values has been updated for nearest_warehouse coordinates, and season.

In [ ]:
dirty_data.describe(include=['O'])

#### Step 16: Customer happiness<br>
We will check based on compound polarity score obtain from customer review, whether customer is happy or not.<br>
1. Create function polarity, call SentimentIntensityAnalyzer() and initialize the object. Check the polarity score of compund for each review. If it is more than 0.05 customer is happy, and disappointed otherwise.<br>
2. If current value is equal to calculated value, keep current value, else replace it.

In [ ]:
# find the compound polarity score for reviews
def polarity(x):
    # initilize object for polarity score
    sid = SentimentIntensityAnalyzer()
    # If review is None keep current happy customer value
    if ('none' in x['latest_customer_review'] or 'None' in x['latest_customer_review']) and len(x['latest_customer_review'].split()) <= 2:
        return x['is_happy_customer']
    # get the polarity score
    score = sid.polarity_scores(x['latest_customer_review'])
    # if score is >= 0.05, customer is happy
    if score['compound'] >= 0.05:
        polarity = True
        if x['is_happy_customer'] is True: # if already True, keep current value
            return x['is_happy_customer']
        else:
            return polarity # else update with True
    else: # customer is not happy
        polarity = False
        if x['is_happy_customer'] is False: # if already false, keep current value
            return x['is_happy_customer']
        else:
            return polarity # else update with False
        
# apply polarity function on is_happy_customer
dirty_data['is_happy_customer'] = dirty_data.apply(polarity,axis=1) 

 #### Step 17: Order Price Valuation<br>
 It is not possible to evaluate order_price. Since it is not possible to determine which row is correct we cannot evaluate the value of order_price column. So, I am considering that they are correct.

## Save the Cleaned data

In [ ]:
dirty_data.to_csv('30749476_dirty_data_solution.csv',index=False)

## Missing Data

#### Step 18: Summary of Missing Data<br>
Since all the values are correct in this we only concered about missing values. Which we can see below.

In [ ]:
missing_data = pd.read_csv('30749476_missing_data.csv')
missing_data.describe()

In [ ]:
missing_data.describe(include=['O'])

#### Step 19: Total NaN values in each Column

In [ ]:
missing_data.isna().sum()

#### Step 20: Fill Nearest Warehouse.<br>
We will use same method used in **"Step 10: Check correct Nearest Warehouse"**. Only difference here is we return nearest warehouse name only for NaN or null values and existing values in column will remain same.

In [ ]:
def fill_warehouse(x):
    # empty list
    kiloList = [] 
    Radius = 6378 # earth radius
    # customer latitude
    lat2 = radians(x['customer_lat'])
    # customer longitude
    long2 = radians(x['customer_long'])
    # warehouse list
    warehouseList = ['Nickolson','Thompson','Bakers']
    # initiate for loop
    for location in warehouseList:
        # if Nickolson warehouse
        if location == 'Nickolson':
            # get Nickolson coordinates
            lat1 = radians(Nickolson['Nickolson'][0])
            long1 = radians(Nickolson['Nickolson'][1])
            # calculate distance with customer location
            distance = kilometers(lat1,lat2,long1,long2,Radius)
            # append list
            kiloList.append(distance)
        # if Thompson warehouse
        if location == 'Thompson':
            # get Thompson coordinates
            lat1 = radians(Thompson['Thompson'][0])
            long1 = radians(Thompson['Thompson'][1])
             # calculate distance with customer location
            distance = kilometers(lat1,lat2,long1,long2,Radius)
            # append list
            kiloList.append(distance)

        if location == 'Bakers':
            # get Bakers coordinates
            lat1 = radians(Bakers['Bakers'][0])
            long1 = radians(Bakers['Bakers'][1])
            # calculate distance with customer location
            distance = kilometers(lat1,lat2,long1,long2,Radius)
            # append list
            kiloList.append(distance)
    # minimum value from the list
    min_distance = min(kiloList)
    # get index of minimum value
    ind = kiloList.index(min_distance)
    # get the warehouse based on the index found through minimum distance
    actual_warehouse = warehouseList[ind]

    if x['nearest_warehouse'] is np.nan:
        return actual_warehouse
    else:
        return x['nearest_warehouse'] 

# apply fill_warehouse function on nearest_warehouse
missing_data['nearest_warehouse'] = missing_data.apply(fill_warehouse, axis = 1)

#### Step 21: Fill distance to nearest warehouse.<br>
Use methodology use in **"Step 15: Check distance to nearest warehouse."**. Only difference here is we return distance to nearest warehouse name only for NaN or null values and existing values in column will remain same.

In [ ]:
def fill_distance(x):
    Radius = 6378 # Earth Radius
    if x['nearest_warehouse'] == 'Nickolson':
        # if nearest warehouse is Nickolson, get its coordinate
        lat1 = radians(Nickolson['Nickolson'][0])
        long1 = radians(Nickolson['Nickolson'][1])
        
    if x['nearest_warehouse'] == 'Thompson':
        # if nearest warehouse is Thompson, get its coordinate
        lat1 = radians(Thompson['Thompson'][0])
        long1 = radians(Thompson['Thompson'][1])
    
    if x['nearest_warehouse'] == 'Bakers':
        # if nearest warehouse is Bakers, get its coordinate
        lat1 = radians(Bakers['Bakers'][0])
        long1 = radians(Bakers['Bakers'][1])
    # customer coordinates
    lat2 = radians(x['customer_lat'])
    long2 = radians(x['customer_long'])

    longitude = long2 - long1
    latitude = lat2 - lat1
    # arc calculation
    arc = (sin(latitude/2))**2 + cos(lat1) * cos(lat2) * (sin(longitude/2))**2
    circumfrence = 2 * atan2(sqrt(arc), sqrt(1-arc))
    # actual distance
    actual_distance = Radius * circumfrence
    
    if x['distance_to_nearest_warehouse'] == 0: # if distance is zero, upadte the distance value
        return round(actual_distance,4)
    else:
        return x['distance_to_nearest_warehouse'] # else keep the current distance

In [ ]:
# replace all np.nan values with zero, as column is float and it's easy to read data with reolaced value
missing_data['distance_to_nearest_warehouse'] = missing_data['distance_to_nearest_warehouse'].replace({np.nan:0})
# apply fill_distance function on distance_to_nearest_warehouse
missing_data['distance_to_nearest_warehouse'] = missing_data.apply(fill_distance, axis = 1)

#### Step 22: Fill happy Customer values.<br>
Use same methodology used in **"Step 16: Customer happiness"**. Only difference here is we return happiness value (True or False) only for NaN or null values and existing values in column will remain same.

In [ ]:
# Convert 1, 0 to True and False boolean respectively
missing_data['is_happy_customer'] = missing_data['is_happy_customer'].replace({1.0:True, 0.0:False})

In [ ]:
missing_data[missing_data['is_happy_customer'].isna()] # get the dataframe with NaN values in is_happy_customer

In [ ]:
# replace the NaN values with string 'None' for better analysis
missing_data['is_happy_customer'] = missing_data['is_happy_customer'].replace({np.nan:'None'})

In [ ]:
def polarity(x):
    sid = SentimentIntensityAnalyzer() # initilize object for polarity score
    score = sid.polarity_scores(x['latest_customer_review']) # get the polarity score
#       if score is >= 0.05, customer is happy
    if score['compound'] >= 0.05:
        if x['is_happy_customer'] == 'None': # Replace the empty cell with the value True
            return True
        else:
            return x['is_happy_customer'] # else if cell is not empty keep current value
    else: # customer is not happy
        if x['is_happy_customer'] == 'None': # Replace the empty cell with the value True
            return False
        else:
            return x['is_happy_customer'] # else if cell is not empty keep current value

In [ ]:
# apply polarity function on is_happy_customer column and replace empty cells with calculated boolean value
missing_data['is_happy_customer'] = missing_data.apply(polarity,axis=1)

In [ ]:
# getting all the brands in shopping cart in a list
cart_list = []
cart_dict = {}
for items in dirty_data['shopping_cart']:
    for item in eval(items):
        cart_list.append(item[0])
cartItems = set(cart_list)
for x in cartItems:
    cart_dict[x] = 0

#### Step 23: Fill the Order Price.<br>
To calculate the order price from items in cart we will use following steps:<br>
1. Make a copy of missing_data.<br>
2. Create new column **"two_Items_shopping_cart"** in copied dataframe. Having only those carts which have 2 branded items and replace remaining cart with NaN.<br>
3. Now, remove all the NaN rows.<br>
4. Sort all the itmes in the Shopping cart with branded item name.<br>
5. Now sort the dataframe based on Shopping Cart column.<br>
6. Drop duplicate values in shopping cart and reset the index.<br>
7. Initiate a while loop to go through each cell of shopping cart.<br>
8. Take the items in current row and compare it with the items of next row. If items in both rows are similar move to step 10.<br>
9. If items are not similar, increment i and move to step 8 again.<br>
10. Save items of current and next row in two list and get there price value.<br>
11. Create an array with quantity of items from current and next row, and similarly for price.<br>
12. Perform linear algebra using np.linalg.solve().<br>
13. Store the item and its price in a dictionary. Check if item already in cart or not. Add only if not exists.<br>
14. Repeat step from 8 to 13 until reach the condition where value of i is equal to length of dataframe. If yes exit the while loop.

In [ ]:
missing_data_copy = missing_data.copy() # make a copy of missing_data
# Get all the cart with two brand items, and make other cart list as NaN
missing_data_copy['two_Items_shopping_cart'] = missing_data_copy['shopping_cart'].apply(lambda x: x if len(eval(x))==2 else np.nan)
# remove the rows with NaN values
missing_data_copy.dropna(axis = 0, inplace = True)

In [ ]:
# defining a function to sort the shopping cart on the basis of branded items
def sort_items(x):
    # convert the string of list to list
    items = eval(x['shopping_cart'])
    # sort the list based on first element of tuples
    items.sort(key = lambda x: x[0])
    # return string of list
    return str(items)

# Sort the elements on the shopping cart
missing_data_copy['shopping_cart'] = missing_data_copy.apply(sort_items, axis = 1)

In [ ]:
# Sort datafrmae based on shopping cart
missing_data_copy.sort_values(by=['shopping_cart'], inplace=True)
# drop all the duplicates to satisfy linear algebra equation
missing_data_copy.drop_duplicates(subset=['shopping_cart'], inplace=True)
# reseting the index
missing_data_copy = missing_data_copy.reset_index()

In [ ]:
price_dict = {} # define an empty
# initialize i
i = 0
while True:
    # convert the string of list to list
    item_list = eval(missing_data_copy.loc[i,'shopping_cart'])
    # get the items in cart for first list
    item = [item_list[0][0], item_list[1][0]]
    # get the second row of shopping cart
    item_list_next = eval(missing_data_copy.loc[i+1,'shopping_cart'])
    # get the items in next row of cart
    item_next = [item_list_next[0][0], item_list_next[1][0]]

    # if items in current row also exists in next row
    if item[0] in item_next and item[1] in item_next:
        # get the price for current row
        price = (missing_data_copy.loc[i,'order_price'])
        # get price for next row
        price_next = missing_data_copy.loc[i+1,'order_price']
        # creat array for linear algebra equation
        a = np.array([[int(item_list[0][1]), int(item_list[1][1])], [int(item_list_next[0][1]),int(item_list_next[1][1])]])
        # provide the price
        b = np.array([price,price_next])
        # price for items in shopping cart
        x = np.linalg.solve(a, b)
        # if item does not exist in dictionary, add with their respective price
        if item[0] not in price_dict:
            price_dict[item[0]] = x[0]
        if item[1] not in price_dict:
            price_dict[item[1]] = x[1]
            
        i+=1
    else:
        i+=1
    # stop condition for while loop, once i reach value of length of dataframe
    if i == missing_data_copy.shape[0]-1:
        break

In [ ]:
# price of each branded item
price_dict

#### Step 24: Fill Order Price.<br>
1. Amount calculated in above step 23 for each item. Now we will calcuate the order price based on quantity of each item.<br>
2. Convert the string of list to List. Initiate price=0.<br>
3. Take the price of item from price_dict dictionary and multiply with it's quantity. Similarly do for all the items in that cart and add them together.<br>
4. Replcae only NaN values and keep the current values.

In [ ]:
# replace NaN value s with 0, as column is of type float
missing_data['order_price'] = missing_data['order_price'].replace({np.nan:0})

def fill_price(x):
    # initialize price
    price = 0
    # get items in shopping cart as a list
    items = eval(x['shopping_cart'])
    for tup in items:
        price += price_dict[tup[0]] * int(tup[1]) # multiply item price with quantity and increment price
    if x['order_price'] == 0: # if cell is 0, update calculated price
        return price
    else:
        return x['order_price'] # else return current price value

In [ ]:
missing_data['order_price'] = missing_data.apply(fill_price, axis=1) # apply fill price function on order_price column

#### Step 25: Fill delivery Charges<br>
1. Take copy of missing data and select 'delivery_charges','season','distance_to_nearest_warehouse','is_expedited_delivery','is_happy_customer' columns only.<br>
2. Now we will perform the linear regression to predict the delivery charges based on columns selected in above step.<br>
3. Since we have categorical variable in dataset, we will convert them to numeric values using get_dummies() and store in dataframe.<br>
4. Now take copy of above data frame, now we will make prediction on this copied dataframe.<br>
5. Drop NaN from dataframe created in step 3.<br>
6. Initialize liner model object.<br>
7. Fit the model on dataframe created in step 3, excpect on the column delivery_charges as it is our target variable. Model will be fit on target variable delivery_charges.<br>
8. make predictioin on dataframe created in step 4 and save it in main dataframe missing_data as predicted value.<br>
9. Now we will replace the NaN using the predicted value.<br>
10. If value already exist in cell, keep it, else replace it with predicted delivery_charges values.

In [ ]:
df_missing = missing_data.copy() # make a copy of missing data
# get the required columns for prediction of delivery charges
df_missing = df_missing[['delivery_charges','season','distance_to_nearest_warehouse','is_expedited_delivery','is_happy_customer']]

In [ ]:
# apply get_dummies, as season is a categorical column, it will assign numeric values to categorical columns
df_missing = pd.get_dummies(data=df_missing, drop_first=True)
# make copy of the new dataframe
prediction_data = df_missing.copy()
# drop all the NaN rows from the data frame
df_missing.dropna(subset=['delivery_charges'],axis=0,inplace=True)
# Initialize Linear Regression object
LR = LinearRegression()
# Fit the data on df_missing dataframe
LR.fit(df_missing[[x for x in df_missing.columns if x != 'delivery_charges']],df_missing['delivery_charges'])

In [ ]:
# len(LR.predict(missing_data_impute.drop(['delivery_charges'],axis=1)))
# Predict the delivery_charges on prediction_data, and add the predicted value in missing data frame
missing_data['delivery_charges_predicted'] = LR.predict(prediction_data.drop(['delivery_charges'],axis=1))

In [ ]:
# replace the NaN values with 0 for better calculation
missing_data['delivery_charges'] = missing_data['delivery_charges'].replace({np.nan:0})

def fill_charges(x):
    # if cell is zero replace it with predicted value
    if x['delivery_charges'] == 0:
        return x['delivery_charges_predicted']
    else: # else keep the current value
        return x['delivery_charges']


In [ ]:
# apply fill_charges function on delivery_charges columns to fill the missing values
missing_data['delivery_charges'] = missing_data.apply(fill_charges, axis=1)
# drop delivery_charges_predicted column as it's not required
missing_data.drop('delivery_charges_predicted', axis=1, inplace=True)

#### Step 26: Fill Total Order Price.<br>
We will use same methodology we use on **"Step 12: Check Total Order Price"**. Only replace the NaN values and keep the current values.

In [ ]:
# replace the NaN values with 0 for better conditions
missing_data['order_total'] = missing_data['order_total'].replace({np.nan:0})

def fill_total(x):
    # Calculate the total order price
    order_total = x['order_price']*(1-x['coupon_discount']/100)+x['delivery_charges']
    
    if x['order_total'] == 0: # if order price is zero, replace it with calcuated total order price
        return order_total
    else:
        return x['order_total'] # else keep the current value of cell

# apply fill_total on order_total column 
missing_data['order_total'] = missing_data.apply(fill_total, axis=1)

## Check the NaN values

In [ ]:
missing_data.info()

## Save the filled Missing Data

In [ ]:
missing_data.to_csv('30749476_missing_data_solution.csv',index=False)

### Outlier Data

In [ ]:
outlier_data = pd.read_csv('30749476_outlier_data.csv') # Read Outlier Data

#### Step 27: Summary of Numeric columns

In [ ]:
outlier_data.describe() # Analyze the Numeric columns

#### Step 28: Plot Box plot to check Outliers for delivery_charges Column

In [ ]:
plt.figure(num=1, figsize=(25, 10), dpi=150, facecolor='g', edgecolor='k') # size of plot setting
bp = outlier_data.boxplot(column='delivery_charges') # box plot for delivery_charges column

#### Step 29: Use Interquartile to detect Outliers<br>
1. First get the IQR value by subtracting Q3 and Q1 percentile value.<br>
2. Save the summmary of numeric col in dataframe.<br>
3. Select quartile value from 75% and 25%.<br>
4. Calculate upper and lower range by adding 1.5xIQR to 75% precentile and subtracting it from 25% percentile respectively.<br>
5. Now filter the Outlier data based on upper and lower range.

In [ ]:
# Calculate IQR for delivery_charges columns
Q3, Q1 = np.percentile(outlier_data['delivery_charges'], [75 ,25])
iqr = Q3 - Q1

df = outlier_data.describe()

quartile_75 = df.loc['75%','delivery_charges']

quartile_25 = df.loc['25%','delivery_charges']

Upper_Range = quartile_75 + 1.5*iqr
Lower_Range = quartile_25 - 1.5*iqr

outlier_removed = outlier_data[outlier_data['delivery_charges'] >= Lower_Range]
outlier_removed = outlier_removed[outlier_removed['delivery_charges'] <= Upper_Range]


## Save the data after removing Outlier

In [ ]:
outlier_removed.to_csv('30749476_outlier_data_solution.csv',index=False)